In [2]:
import pandas as pd
import geopandas as gpd
from gotrackit.map.Net import Net
from gotrackit.MapMatch import MapMatch
from gotrackit.model.Para import ParaGrid

In [ ]:

if __name__ == '__main__':
    # 1.读取GPS数据
    gps_df = pd.read_csv(r'./product_data/gps_data.csv')
    # gps_df['time'] = pd.to_datetime(gps_df['time'], format='%Y/%m/%d %H:%M:%S')
    gps_df['time'] = gps_df['time'].str.replace('T', ' ').str.replace('Z', '').str.replace('/', '-')
    print(gps_df.head())
    # 2.构建一个net
    link = gpd.read_file(r'./product_data/roads/roads.shp')
    node = gpd.read_file(r'./product_data/nodes/nodes.shp')
    my_net = Net(link_gdf=link,
                 node_gdf=node,
                 fmm_cache=True, fmm_cache_fldr=r'./product_data', recalc_cache=True,
                 cut_off=1400.0,
                 cache_slice=6)
    my_net.init_net()  # net初始化

    # 3. 匹配
    mpm = MapMatch(net=my_net, gps_buffer=185, flag_name='xa_sample',
                   use_sub_net=True, use_heading_inf=True, omitted_l=6.0,
                   del_dwell=True, dwell_l_length=13.0, dwell_n=4,
                   export_html=False, export_geo_res=False, use_gps_source=False,
                   export_all_agents=True,
                   out_fldr=r'./product_data/match_results', dense_gps=False,
                   gps_radius=15.0, is_lower_f = False, dense_interval=110, beta=3, gps_sigma=17,top_k=35)
    # 这里使用 multi_core_execute 方法
    # match_res, may_error_info, error_info = mpm.execute(gps_df=gps_df)
    match_res, may_error_info, error_info = mpm.multi_core_execute(gps_df=gps_df, core_num=6)
    # 输出结果
    print(match_res)
    match_res.to_csv(r'./product_data/match_results/match_result.csv', encoding='utf_8_sig', index=False)
